# AutoGen Function Calling

This tutorial will redo the day-3-function-calling-with-llamaIndex.ipynb tutorial using AutoGen.

The tutorial demonstrates how to use AutoGen to create agents that translate natural language queries into SQL. We'll focus on:

1. Designing effective agents for SQL translation
2. Setting up proper agent interactions
3. Using function calling to execute database operations
4. Creating natural conversations about data

First install the autogen-agentchat package.
```
%pip install autogen-agentchat~=0.2
```

## Basic Setup

First, set up the llm connection and sql connection.

In [1]:
import os
from dotenv import load_dotenv
import sqlite3

load_dotenv()

llm_list = [
    {
        "model": "gpt-4o",
        "api_key": os.getenv("AZURE_OPENAI_KEY"),
        "api_version": os.getenv("OPENAI_API_VERSION"),
        "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
    }
]

llm_config = {
    "temperature": 0.7,
    "config_list": llm_list,
    "timeout": 60,
}

# set up the sql connection
db_file = "sample.db"
db_conn = sqlite3.connect(db_file)